In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.metrics import mean_squared_error

In [2]:
# train = pd.read_csv("train_subset_2.csv")

In [4]:
train = pd.read_csv("test.csv")

In [ ]:
# train = test

In [5]:
train.head(100)

,ego_id,u,v,t,x1,x2,x3
0,8,20,19,185.7,3.839089e-04,0.000000,0.0
1,8,131,125,161.4,4.034464e-01,0.000000,0.0
2,8,73,56,127.0,8.554643e-05,0.000000,0.0
3,8,0,4,594.5,2.886418e-01,0.000000,0.0
4,8,63,73,127.0,4.281692e-07,0.000000,0.0
...,...,...,...,...,...,...,...
95,8,4,8,NaN,4.670412e-02,0.000000,0.0
96,8,106,90,475.2,1.098381e-01,0.000000,0.0
97,8,103,8,594.5,NaN,0.000000,0.0
98,8,126,41,480.5,1.923058e+00,2.944439,0.0


In [6]:
train.shape

(40548780, 7)

In [7]:
attr = pd.read_csv("attr.csv")

In [8]:
attr

,ego_id,u,age,city_id,sex,school,university
0,0,227,68,-1,1,778293348,-1
1,0,45,38,237065842,1,82803468,238500268
2,0,142,60,237065842,1,196560139,-1
3,0,280,66,-1,2,963209731,720783270
4,0,41,18,-1,2,308862409,-1
...,...,...,...,...,...,...,...
14930743,1709396984692,2,16,492149712,2,769209871,-1
14930744,1709396984692,12,15,-1,1,-1,-1
14930745,1709396984692,18,23,-1,1,-1,-1
14930746,1709396984692,4,16,650683235,1,-1,-1


In [9]:
train_2 = pd.merge(train, attr, on=['ego_id', 'u']).merge(attr, right_on=['ego_id', 'u'], left_on=['ego_id', 'v'])

In [10]:
train_2 = train_2.drop_duplicates()

In [11]:
train_2

,ego_id,u_x,v,t,x1,x2,x3,age_x,city_id_x,sex_x,school_x,university_x,u_y,age_y,city_id_y,sex_y,school_y,university_y
0,8,20,19,185.7,3.839089e-04,0.000000,0.0,37,979281502,2,783569353,372487423,19,37,979281502,2,160546731,657399801
1,8,10,19,158.3,1.046045e+00,0.000000,0.0,-1,979281502,2,-1,-1,19,37,979281502,2,160546731,657399801
2,8,2,19,594.5,7.529229e-01,0.000000,0.0,36,979281502,2,160546731,845825535,19,37,979281502,2,160546731,657399801
3,8,8,19,NaN,1.751368e-06,0.000000,0.0,121,-1,1,-1,-1,19,37,979281502,2,160546731,657399801
4,8,20,2,164.1,3.034230e-13,0.000000,0.0,37,979281502,2,783569353,372487423,2,36,979281502,2,160546731,845825535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35484660,1709396984676,1,7,219.2,4.164250e-05,0.000000,0.0,36,-1,1,-1,84847376,7,19,-1,1,-1,-1
35484661,1709396984676,6,7,219.0,1.224247e-02,0.000000,0.0,17,338248086,1,253082810,-1,7,19,-1,1,-1,-1
35484662,1709396984676,48,7,20.6,1.099507e+00,0.000000,1.0,44,-1,1,-1,-1,7,19,-1,1,-1,-1
35484663,1709396984676,48,42,27.1,1.054881e-01,0.000000,0.0,44,-1,1,-1,-1,42,40,-1,2,-1,193142067


In [12]:
def dis(x,y):
    return abs(x-y) if x > 0 and y > 0 else -1

In [13]:
un1 = train_2["university_x"].tolist()
un2 = train_2["university_y"].tolist()

In [14]:
def tmp(x, y):
    if x == -1 or y == -1:
        return -1
    return abs(x - y) 


train_2["city_dist"] = list(map(tmp, train_2["city_id_x"], train_2["city_id_y"]))
train_2["university_dist"] = list(map(tmp, train_2["university_x"], train_2["university_y"]))
train_2["sex"] = (train_2["sex_x"] + train_2["sex_y"]) % 2
train_2["school_dist"] = list(map(tmp, train_2["school_y"], train_2["school_x"]))
train_2["age_dist"] = abs(train_2["age_x"] - train_2["age_y"])
train_2["age_mean"] = (train_2["age_x"] + train_2["age_y"]) / 2



In [ ]:
def sex_cat(x, y):
    if  x == 1 and y == 1:
        return 1
    elif x == 2 and y == 2:
        return 2
    else:
        return 0
    
def is_the_same(x, y):
    if  x == y and x != -1:
        return 2
    elif x != -1 or y != -1:
        return 0
    else:
        return 1
    
def age_eps(x):
    if x <= 8:
        return 1
    else: 
        return 0


train_2['sex2'] = list(map(sex_cat, train_2["sex_x"], train_2["sex_y"]))
train_2["city"] = list(map(is_the_same, train_2["city_id_x"], train_2["city_id_y"]))
train_2["university"] = list(map(is_the_same, train_2["university_x"], train_2["university_y"]))
train_2["school"] = list(map(is_the_same, train_2["school_y"], train_2["school_x"]))
train_2["gen"] = list(map(age_eps, train_2["age_dist"]))

In [ ]:
train_3 = train_2.drop(["city_id_x", "city_id_y", "university_x", "university_y", "sex_x", "sex_y", "school_x", "school_y", "age_x", "age_y", "u_y"], axis=1)

In [ ]:
train_3.rename(columns= {"u_x": "u"} , inplace=True)


In [ ]:
grouped_data1 = train_3.groupby(['ego_id', 'u']).agg({'v':'count'}).reset_index().rename(columns = {'v':'v_count'})

In [ ]:
grouped_data = train_3.groupby(['ego_id', 'v']).agg({'u':'count'}).reset_index().rename(columns = {'u':'u_count'})

In [ ]:
grouped_data = pd.merge(grouped_data1, grouped_data, left_on = ['ego_id', 'u'], right_on = ['ego_id', 'v'])

In [ ]:
grouped_data['freinds'] = grouped_data['v_count']+grouped_data['u_count']

,ego_id,u,v_count,v,u_count,freinds
0,8,0,97,0,75,172
1,8,1,11,1,8,19
2,8,2,14,2,8,22
3,8,3,10,3,14,24
4,8,4,6,4,8,14
...,...,...,...,...,...,...
3284958,1709396984676,105,7,105,4,11
3284959,1709396984676,106,6,106,5,11
3284960,1709396984676,107,4,107,7,11
3284961,1709396984676,109,3,109,2,5


In [ ]:
df = pd.merge(train_3, grouped_data[['ego_id', 'u', 'freinds']], on = ['ego_id', 'u'])
df = pd.merge(df, grouped_data[['ego_id', 'v', 'freinds']], on = ['ego_id', 'v'])
df = df.rename(columns = {'freinds_x':'freinds_u', 'freinds_y':'freinds_v'})

In [ ]:
df['u_v'] = (1 + df['x2'])*df['freinds_u'] + (1 + df['x3'])*df['freinds_v']

In [ ]:
df

,ego_id,u,v,t,x1,x2,x3,city,university,sex,school,age_dist,age_mean,freinds_u,freinds_v,u_v
0,13,26,20,256.7,6.667426e-09,1.000000,2.0,0,-1,0,-1,2,29.0,32,32,96.000000
1,13,0,20,144.7,1.536793e-04,1.000000,2.0,0,-1,0,0,5,32.5,260,32,324.000000
2,13,148,20,NaN,0.000000e+00,1.000000,2.0,0,-1,0,0,19,39.5,16,32,80.000000
3,13,2,20,79.1,6.092806e-07,1.000000,1.0,18712486,-1,0,84863200,5,32.5,22,32,54.000000
4,13,17,20,338.1,1.138490e-08,1.000000,2.0,0,-1,0,0,8,34.0,44,32,108.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26006041,1709396984692,12,2,9.5,7.487368e-01,1.000000,1.0,-1,-1,1,-1,1,15.5,6,16,22.000000
26006042,1709396984692,15,2,7.5,1.835345e+00,3.484907,1.0,-1,-1,0,378527603,4,18.0,10,16,50.849066
26006043,1709396984692,0,18,NaN,0.000000e+00,3.484907,1.0,-1,-1,1,-1,4,21.0,28,2,99.577386
26006044,1709396984692,0,10,0.9,1.993434e+00,3.302585,1.0,-1,-1,1,-1,3,17.5,28,8,100.472383


In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rgrs = RandomForestRegressor(max_depth = 5, random_state=42)

In [ ]:
rgrs.fit(df[['x2', 'x3', 'sex', 'city', 'university', 'age_mean', 'u_v', 'school', 'age_dist']], df['x1'])

RandomForestRegressor(max_depth=5, random_state=42)

In [56]:
pred=rgrs.predict(df[['x2', 'x3', 'sex', 'city', 'university', 'age_mean', 'u_v', 'school', 'age_dist']])

In [57]:
from sklearn.metrics import mean_squared_error
mean_squared_error(df['x1'], pred)

1.0201155539899025

In [60]:
import pickle

In [62]:
# Предположим, что у вас есть объект модели с именем model
# Сохраняем модель на диск
with open('model.pkl', 'wb') as f:
    pickle.dump(rgrs, f)